In [1]:
import os
import numpy as np
from PIL import Image, ImageFile
from skimage import io
import matplotlib.pyplot as plt
import tifffile
from scipy import misc
from tifffile import TiffFile
from tifffile.tifffile import TIFF_DECOMPESSORS
from visualization import ids_to_greyscale, load_depth, show_bounding_boxes, load_stencil_ids, load_stencil_flags, \
get_bounding_boxes, show_loaded_bounding_boxes, get_detections
from math import tan, atan, radians, degrees

In [2]:
import visualization
visualization.multi_page = False
visualization.ini_file = 'gta-postprocessing.ini'

conn = visualization.get_connection()
cur = conn.cursor()
cur.execute("""SELECT snapshot_id, imagepath, cam_near_clip, cam_far_clip, timestamp, view_matrix, proj_matrix, world_matrix, \
    width, height, camera_fov, ui_width, ui_height,  \
    ARRAY[st_x(camera_relative_rotation), st_y(camera_relative_rotation), st_z(camera_relative_rotation)] as relative_cam_rot, \
    ARRAY[st_x(camera_pos), st_y(camera_pos), st_z(camera_pos)] as camera_pos \
    FROM snapshots \
    ORDER BY snapshot_id DESC \
    LIMIT 1 \
    OFFSET 3 \
    """)

results = [dict(res) for res in cur]
for i, res in enumerate(results):
    res['view_matrix'] = np.array(res['view_matrix'])
    res['proj_matrix'] = np.array(res['proj_matrix'])
    res['world_matrix'] = np.array(res['world_matrix'])
    res['camera_pos'] = np.array(res['camera_pos'])
    results[i] = res

res = results[0]
name = res['imagepath']
print(name)

2018-03-07--18-50-24--246


In [ ]:
%matplotlib inline
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.axis('off')
plt.imshow(im)
plt.show()

In [ ]:
detections = get_detections(name)
print('total detections: {}'.format(len(detections)))
detections = [d for d in detections if np.linalg.norm(res['camera_pos'] - d['pos']) < 16]
print('near detections: {}'.format(len(detections)))

In [ ]:
for d in detections:
    print('pos: {}, type: {}, class: {}, handle: {}, dist. {}'.format(d['pos'], d['type'], d['class'], d['handle'], np.linalg.norm(res['camera_pos'] - d['pos'])))


In [ ]:
l_car = detections[0]
r_car = detections[3]

In [3]:
width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

view_matrix = res['view_matrix']
proj_matrix = res['proj_matrix']
l_vec_pos = l_car['pos']
r_vec_pos = r_car['pos']
l_vec = np.array([l_vec_pos[0], l_vec_pos[1], l_vec_pos[2], 1])
r_vec = np.array([r_vec_pos[0], r_vec_pos[1], r_vec_pos[2], 1])

print('original points')
print(l_vec)
print(r_vec)

l_vec_camera = view_matrix @ l_vec
r_vec_camera = view_matrix @ r_vec
l_vec_camera /= l_vec_camera[3]
r_vec_camera /= r_vec_camera[3]

print('camera points')
print(l_vec_camera)
print(r_vec_camera)

l_vec_ndc = proj_matrix @ l_vec_camera
r_vec_ndc = proj_matrix @ r_vec_camera
l_vec_ndc /= l_vec_ndc[3]
r_vec_ndc /= r_vec_ndc[3]

print('projected points')
print(l_vec_ndc)
print(r_vec_ndc)


NameError: name 'l_car' is not defined

In [ ]:
def pixel_to_ndc(pixel, size):
    p_y, p_x = pixel
    s_y, s_x = size
    return ((2/s_y)*p_y - 1, (2/s_x)*p_x - 1)

def normalized_to_pixel(pixel, size):
    p_y, p_x = pixel
    s_y, s_x = size
    return ((s_y/2)*p_y + (s_y/2), (s_x/2)*p_x + (s_x/2))

In [ ]:
l_vec_point = normalized_to_pixel((l_vec_ndc[1], l_vec_ndc[0]), size)
r_vec_point = normalized_to_pixel((r_vec_ndc[1], r_vec_ndc[0]), size)

print('projected points')
print(l_vec_point)
print(r_vec_point)


In [ ]:
%matplotlib inline
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.axis('off')
plt.plot([l_vec_point[1], r_vec_point[1]], [l_vec_point[0], r_vec_point[0]], 'o')
plt.imshow(im)
plt.show()

### mapping all 4 cars

In [ ]:
vecs_pos = [d['pos'] for d in detections]
vecs = np.array([[vec[0], vec[1], vec[2], 1] for vec in vecs_pos]).T

width = res['width']
height = res['height']
size = (height, width)
near_clip = res['cam_near_clip']
far_clip = res['cam_far_clip']

view_matrix = res['view_matrix']
proj_matrix = res['proj_matrix']

print('original points')
print(vecs)

vecs_camera = view_matrix @ vecs
vecs_camera /= vecs_camera[3, :]

print('camera points')
print(vecs_camera)

vecs_ndc = proj_matrix @ vecs_camera
vecs_ndc /= vecs_ndc[3, :]

print('projected points')
vecs_ndc[1, :] *= -1 
vecs_ndc[0, :] *= -1 
print(vecs_ndc)

points = []
for vec in vecs_ndc.T:
    points.append(normalized_to_pixel((vec[1], vec[0]), size))
points = np.array(points)
points = points.astype(int)
    
print('pixel points')
print(points)


In [ ]:
%matplotlib inline
im = Image.open(os.path.join(visualization.get_in_directory(), name + '.tiff'))
fig = plt.figure(figsize=(12,12))
plt.axis('off')
plt.plot(points[:, 1], points[:, 0], 'o')
plt.imshow(im)
plt.show()

In [ ]:
depth = load_depth(name)
print("depth points")
print("expected:")
print(vecs_ndc[2, :])
print("real:")
print(depth[(points[:, 0], points[:, 1])])

In [ ]:
%matplotlib notebook
depth = load_depth(name)
fig = plt.figure(figsize=(12,12))
plt.axis('on')
plt.imshow(depth, cmap='gray')

In [42]:
def construct_proj_matrix():
    fov = 50.0
    # for z coord
    f = 1.5  # the near clip, but f in the book
    n = 10003  # the far clip, somewhere in the ned of the world
    # x coord
    H = 1080
    W = 1914
    r = W*n*tan(radians(fov)/2)/H
    l = -r
    # y coord   
    t = n*tan(radians(fov)/2)
    b = -t
    #x00 = 2*n/(r-l)
    x00 = H/(tan(radians(fov)/2)*W)
    #x11 = 2*n/(t-b)
    x11 = 1/tan(radians(fov)/2)
    return np.array([        
        [x00, 0, -(r+l)/(r-l), 0],
        [0, x11, -(t+b)/(t-b), 0],
        [0, 0, -f/(f-n), -f*n/(f-n)],
        [0, 0, -1, 0],
    ])

constructed = construct_proj_matrix()
print(proj_matrix)
print(constructed)
print(constructed - proj_matrix)
print(np.abs(constructed - proj_matrix) > 1e-6)

[[  1.21006652e+00   4.02395929e-11   5.23221372e-09   2.33362242e-06]
 [  3.86116095e-10   2.14450701e+00   5.74842176e-10  -2.87629736e-06]
 [  8.21432196e-13  -2.34242218e-14   1.49965293e-04   1.50022490e+00]
 [  9.30889209e-18  -8.67361738e-19  -1.00000000e+00   1.42108547e-14]]
[[  1.21006660e+00   0.00000000e+00  -0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   2.14450692e+00  -0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.49977503e-04   1.50022497e+00]
 [  0.00000000e+00   0.00000000e+00  -1.00000000e+00   0.00000000e+00]]
[[  7.78383344e-08  -4.02395929e-11  -5.23221372e-09  -2.33362242e-06]
 [ -3.86116095e-10  -8.44995416e-08  -5.74842176e-10   2.87629736e-06]
 [ -8.21432196e-13   2.34242218e-14   1.22102182e-08   6.30693917e-08]
 [ -9.30889209e-18   8.67361738e-19   0.00000000e+00  -1.42108547e-14]]
[[False False False  True]
 [False False False  True]
 [False False False False]
 [False False False False]]


In [7]:
x00 = proj_matrix[0, 0]
x11 = proj_matrix[1, 1]
x22 = proj_matrix[2, 2]
x23 = proj_matrix[2, 3]

n = -x23/x22
f = -x23/(x22 - 1)

print('n:', n)
print('f:', f)

n: -10003.8140266
f: 1.5004499186


In [23]:
fov = res['camera_fov']
print('fov: ', fov)
tan(radians(fov))
n = 1.5
t = n*tan(radians(fov)/2)
print('t: ', t)
W = res['width']
H = res['height']
fov_h = fov*W/H
print('fov_h: ', fov_h)
r = n*tan(radians(fov_h)/2)
print('r: ', r)
print('W: ', W)
print('H: ', H)
print('near clip: ', near_clip)
print('x00_c: ', 1/tan(radians(fov_h)/2))
print('x00_c2: ', H/(W*tan(radians(fov)/2)))
print('x11_c: ', 1/tan(radians(fov)/2))
print('x00: ', x00)
print('x11: ', x11)

fov_c_v = degrees(atan(1/x11))*2
print('fov_c_v: ', fov_c_v)
fov_c_h = fov_c_v*W/H
print('fov_c_h: ', fov_c_h)
fov_c_h2 = degrees(atan(1/x00))*2
print('fov_c_h2: ', fov_c_h2)

H=res['height']
#H=res['ui_height']
print('for height {}, width is {}'.format(H, (fov_c_v*H)/fov_c_h2))
print('for height {}, width is {}'.format(H, (tan(radians(fov_c_h2))*H)/tan(radians(fov_c_v))))
print('ui_width', res['ui_width'])
print('width', res['width'])

print('aspect ration from matrix:', x11 / x00)
print('H/W:', H / W)
print('aspect ration from db:', res['width'] / res['height'])
print('aspect ration from ui_db:', res['ui_width'] / res['ui_height'])
print('width by db ratio:', (x11 / x00)*res['height'])


fov:  50.0
t:  0.6994614872324979
fov_h:  88.61111111111111
r:  1.46407266619936
W:  1914
H:  1080
near clip:  1.5
x00_c:  1.0245393105342955
x00_c2:  1.2100666009144845
x11_c:  2.1445069205095586
x00:  1.21006652308
x11:  2.14450700501
fov_c_v:  49.999998270568014
fov_c_h:  88.61110804617331
fov_c_h2:  79.14071211128197
for height 1080, width is 682.3289390709872
for height 1080, width is 4724.048641926896
ui_width 1280
width 1914
aspect ration from matrix: 1.77222240605
H/W: 0.5642633228840125
aspect ration from db: 1.7722222222222221
aspect ration from ui_db: 1.7777777777777777
width by db ratio: 1914.00019854
